## Construction of combined module SMs

In [3]:
from sympy import *

Define the two SMs to be combined

In [ ]:
stoich_matrix_1 = Matrix( [[-1,1,0], [1,-1,-1], [0,0,1], [-1,0,-1], [0,1,0]])
num_internal_species_1 = 3
labels_1 = {0: 'Ea', 1: 'EaS', 2: 'EaS2', 3: 'S', 4: 'Na'}


stoich_matrix_2 = Matrix([[-1, 0, 1, 0, 0], # first row of internal
                           [1, -1, 0, -1, 0], 
                           [0, 1, -1, 0, 1], 
                           [0, 0, 0, 1, -1],
                             [0, 0, 0, -1, 0], # first row external 
                             [0, 0, 0, 0, 1],
                             [-1, 0, 0, 0, 0],
                             [0, 0, 1, 0, 0]])

num_internal_species_2 = 4
labels_2 = {0: 'Eb', 1: 'EbF', 2: 'EbW', 3: 'Eb*', 4: 'Na', 5: 'Nb', 6: 'F', 7: 'W'}



Create class to assign needed properties of modules

In [5]:
class AssignProperties:

    def __init__(self, SM, num_internal_species, species_labels):

        self.matrix = SM
        self.internal_SM = self.matrix[0:num_internal_species, :]
        self.external_SM = self.matrix[num_internal_species: len(SM), :]
        self.labels = dict(species_labels)
        self.num_internal_species = num_internal_species

mod1 = AssignProperties(stoich_matrix_1, 3, labels_1)
mod2 = AssignProperties(stoich_matrix_2, 4, labels_2)


        

In [6]:
mod1.internal_SM

Matrix([
[-1,  1,  0],
[ 1, -1, -1],
[ 0,  0,  1]])

In [7]:
mod2.internal_SM

Matrix([
[-1,  0,  1,  0,  0],
[ 1, -1,  0, -1,  0],
[ 0,  1, -1,  0,  1],
[ 0,  0,  0,  1, -1]])

detect overlap code: From JW

In [57]:

class CombinedSM:
     
    def __init__(self, mod1, mod2):
        
        self.mod1 = mod1
        self.mod2 = mod2

        self.find_matching_species()
        self.find_conjugated_rows()  

        
    def find_matching_species(self):  # adpated from JW
        """
        Find indices of matching external species between mod1 and mod2.
        """

        # Removes all species corresponding to the internal species in the list of species (by taking those after the 
        # no. of rows in the internal SM) and stores the index (from 0) and name of the external species
        # essentially creates a dictionary of external species

        self.mod1_external_species_list = {
            i: v for i, v in enumerate(list(self.mod1.labels.values())[self.mod1.internal_SM.rows:])
        }
        self.mod2_external_species_list = {
            i: v for i, v in enumerate(list(self.mod2.labels.values())[self.mod2.internal_SM.rows:])
        }
        # print(self.mod1_external_species_list)
        # print(self.mod2_external_species_list)
        # takes the dictionary of external species and finds the indices in each dict at which the species are the same
        # , stores in the list self.matching_species

        self.matching_species = []
        for key1, value1 in self.mod1_external_species_list.items():
            for key2, value2 in self.mod2_external_species_list.items():
                if value1 == value2:
                    self.matching_species.append((key1, key2))

        #  if there ARE overlapping species: convert the new list into two new lists of indexes and species names
        # print(self.matching_species)

        if self.matching_species:
            self.keys_1, self.keys_2 = zip(*self.matching_species)
            self.keys_1 = list(self.keys_1)
            self.keys_2 = list(self.keys_2)
        else:
            self.keys_1, self.keys_2 = [], [] # returns an empty list overwise to prevent errors

        # print("Keys:")
        # print(len(self.keys_1))
        # print(self.keys_2)
        return self.keys_1, self.keys_2

        # So now we know the position of overlapping species in the two dicts of species names defined outside the class



    def find_conjugated_rows(self):
        
        # Find the rows of the two SMs that correspond to the overlapping species determined earlier

        overlapping_species_rows_mod1 = []

        for i in range(len(self.keys_1)):

            overlapping_species_rows_mod1.append((self.mod1.matrix)[self.keys_1[i] + self.mod1.num_internal_species, :])

        # print("overlapping species row for mod1: ")
        # print(overlapping_species_rows_mod1)

        overlapping_species_rows_mod2 = []

        for i in range(len(self.keys_1)):

            overlapping_species_rows_mod2.append((self.mod2.matrix)[self.keys_2[i] + self.mod2.num_internal_species, :])

        # print("overlapping species row for mod2: ")
        # print(overlapping_species_rows_mod2)

        # conjugate the two rows for each overalpping species:

        conjugated_rows = []

        for k in range(len(self.keys_1)):

            conjugated_rows.append(overlapping_species_rows_mod1[k].row_join(overlapping_species_rows_mod2[k]))
        
        self.conjugated_rows = conjugated_rows

        return conjugated_rows


    def construct_new_SM(self):

        num_cols_new_SM = self.mod1.matrix.cols + self.mod2.matrix.cols
        

        # Create block 0 matrices for the internal part of the new SM

        upper_right_internals = zeros(self.mod1.num_internal_species, num_cols_new_SM - self.mod1.matrix.cols)
        lower_left_internals = zeros(self.mod2.num_internal_species, num_cols_new_SM - self.mod2.matrix.cols)

        # Create block 0 matrices for the external part of the new SM

        upper_right_externals = zeros(self.mod1.matrix.rows - self.mod1.num_internal_species - len(self.conjugated_rows)\
                                           , num_cols_new_SM - self.mod1.matrix.cols)
        
        lower_left_externals = zeros(self.mod2.matrix.rows - self.mod2.num_internal_species - len(self.conjugated_rows),\
                                           num_cols_new_SM - self.mod2.matrix.cols)

        
        # Use these to construct the new SM block by block

        new_SM_line_1 = (self.mod1.internal_SM.row_join(upper_right_internals))

        # The line that contains the conjugated rows goes here
        #========================================
        new_SM_line_2 = self.conjugated_rows[0]
        
        for vector in self.conjugated_rows[1:]:
            new_SM_line_2 = new_SM_line_2.col_join(vector)
            # print("Block containing overlapping species:")
            # print(new_SM_line_2)
        #========================================
    
        new_SM_line_3 = lower_left_internals.row_join(self.mod2.internal_SM)

        new_SM_line_4 = self.mod1.external_SM[[i for i in range(self.mod1.external_SM.rows) if i not in self.keys_1], :].row_join(\
            upper_right_externals)
        
        new_SM_line_5 = (lower_left_externals).row_join(\
            self.mod2.external_SM[[i for i in range(self.mod2.external_SM.rows) if i not in self.keys_2], :])
        
        # Final SM

        new_SM = new_SM_line_1.col_join(new_SM_line_2).col_join(new_SM_line_3).col_join(new_SM_line_4).col_join(new_SM_line_5)


        return new_SM






        
        

# Test with modules 1,2 from 2025 paper: WORKS

In [60]:
stoich_matrix_1 = Matrix( [[-1,1,0], [1,-1,-1], [0,0,1], [-1,0,-1], [0,1,0]])
num_internal_species_1 = 3
labels_1 = {0: 'Ea', 1: 'EaS', 2: 'EaS2', 3: 'S', 4: 'Na'}


stoich_matrix_2 = Matrix([[-1, 0, 1, 0, 0], # first row of internal
                           [1, -1, 0, -1, 0], 
                           [0, 1, -1, 0, 1], 
                           [0, 0, 0, 1, -1],
                             [0, 0, 0, -1, 0], # first row external 
                             [0, 0, 0, 0, 1],
                             [-1, 0, 0, 0, 0],
                             [0, 0, 1, 0, 0]])

num_internal_species_2 = 4
labels_2 = {0: 'Eb', 1: 'EbF', 2: 'EbW', 3: 'Eb*', 4: 'Na', 5: 'Nb', 6: 'F', 7: 'W'}



In [62]:
mod1 = AssignProperties(stoich_matrix_1, num_internal_species_1, labels_1)
mod2 = AssignProperties(stoich_matrix_2, num_internal_species_2, labels_2)

l = CombinedSM(mod1,mod2)
l.construct_new_SM()

Matrix([
[-1,  1,  0,  0,  0,  0,  0,  0],
[ 1, -1, -1,  0,  0,  0,  0,  0],
[ 0,  0,  1,  0,  0,  0,  0,  0],
[ 0,  1,  0,  0,  0,  0, -1,  0],
[ 0,  0,  0, -1,  0,  1,  0,  0],
[ 0,  0,  0,  1, -1,  0, -1,  0],
[ 0,  0,  0,  0,  1, -1,  0,  1],
[ 0,  0,  0,  0,  0,  0,  1, -1],
[-1,  0, -1,  0,  0,  0,  0,  0],
[ 0,  0,  0,  0,  0,  0,  0,  1],
[ 0,  0,  0, -1,  0,  0,  0,  0],
[ 0,  0,  0,  0,  0,  1,  0,  0]])

# Test with a different two modules which overlap at 2 species: WORKS

In [69]:

SM_1 = Matrix([[-1,1,0],[1,-1,-1],[-1,0,-1],[0,0,1],[0,1,0]])
internals_1 = 2
SM_2 = Matrix([[-1,0,1],[1,-1,0],[0,1,-1],[-1,0,0],[0,0,-1],[0,1,0]])
internals_2 = 3

labels_1_new = {0: 'B', 1: 'C', 2: 'A', 3: 'D', 4: 'E'}
labels_2_new = {0: 'F', 1: 'G', 2: 'H', 3: 'D', 4: 'E', 5:'I'}


In [67]:
M1 = AssignProperties(SM_1, internals_1, labels_1_new)

M2 = AssignProperties(SM_2, internals_2, labels_2_new)

In [68]:
n = CombinedSM(M1,M2)

n.construct_new_SM()

Matrix([
[-1,  1,  0,  0,  0,  0],
[ 1, -1, -1,  0,  0,  0],
[ 0,  0,  1, -1,  0,  0],
[ 0,  1,  0,  0,  0, -1],
[ 0,  0,  0, -1,  0,  1],
[ 0,  0,  0,  1, -1,  0],
[ 0,  0,  0,  0,  1, -1],
[-1,  0, -1,  0,  0,  0],
[ 0,  0,  0,  0,  1,  0]])